## XOR with logistic refression

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.loadtxt.html
xy = np.loadtxt('train_data/lab9-1_train.txt', unpack=True)
x_data = np.transpose(xy[0:-1])
y_data = np.reshape(xy[-1], (4,1))

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([1, len(x_data)], -1.0, 1.0))

# Our hypothesis
h = tf.matmul(W, X)
hypothesis = tf.div(1., 1.+tf.exp(-h))
# cost function
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))

# Minimize
a = tf.Variable(0.01) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Before starting, initialize the bariables. We will 'run' this first.
init = tf.global_variables_initializer()

In [3]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # fit the line.
    for step in range(1000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis + 0.5), Y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))

0 0.823534 [[ 0.81494784 -0.79384154 -0.04151426 -0.56615359]]
200 0.73328 [[ 0.81494784 -0.55438042  0.04998901 -0.23518892]]
400 0.706952 [[ 0.81494784 -0.40540081  0.06945921 -0.06673881]]
600 0.698986 [[ 0.81494784 -0.31239125  0.0583694   0.01518091]]
800 0.696104 [[ 0.81494784 -0.2519058   0.03707727  0.05437431]]
[array([[ 0.52186334,  0.46558082]], dtype=float32), array([[ 1.,  0.]], dtype=float32), array([[False,  True],
       [ True, False],
       [ True, False],
       [False,  True]], dtype=bool), 0.5]
Accuracy: 0.5


### Does not work!
우리가 예상한 값은 [0,1,1,0]이지만 결과는 [1,1,0,0]이 나옴  
50% 밖에 맞지 않음 

## NN for XOR

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [5]:
W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2= tf.Variable(tf.zeros([1]), name="Bias2")

# Our hypothesis
L2 = tf.sigmoid(tf.matmul(X, W1) + b1)
hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

In [6]:
# cost function
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))

# Minimize
a = tf.Variable(0.1) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Before starting, initialize the bariables. We will 'run' this first.
init = tf.global_variables_initializer()

In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # fit the line.
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 4000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W1), sess.run(W2))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis + 0.5), Y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))

0 0.708173 [[-0.47884896  0.49346587]
 [-0.29835552  0.97950274]] [[ 0.05120474]
 [ 0.53656435]]
4000 0.497867 [[ 0.43444797  4.90236998]
 [ 0.5924719   4.9158783 ]] [[-0.89916104]
 [ 4.31880331]]
8000 0.0363758 [[ 3.96563601  6.43276787]
 [ 3.96702051  6.44514942]] [[-8.84007454]
 [ 8.4118042 ]]
12000 0.0148474 [[ 4.62015343  6.83183098]
 [ 4.62128401  6.84000921]] [[-10.59400558]
 [  9.88728523]]
16000 0.00922296 [[ 4.92886782  7.04842663]
 [ 4.92984056  7.05493832]] [[-11.49384403]
 [ 10.72148323]]
20000 0.00666375 [[ 5.12687349  7.1954608 ]
 [ 5.12774086  7.2010417 ]] [[-12.10095406]
 [ 11.30376244]]
[array([[ 0.00807264],
       [ 0.99398851],
       [ 0.99398524],
       [ 0.00646607]], dtype=float32), array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]], dtype=float32), array([[ True],
       [ True],
       [ True],
       [ True]], dtype=bool), 1.0]
Accuracy: 1.0


## Wide NN for XOR

In [8]:
# Wide network: Use more neurons in each layer. 
W1 = tf.Variable(tf.random_uniform( [2,10], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [10,1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([10]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

In [9]:
# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
hypothesis = tf.sigmoid( tf.matmul(L2,W2) + b2)

# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.global_variables_initializer()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 4000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.724131 [[ -9.77419913e-01  -4.46508974e-01   8.91232014e-01   4.72538054e-01
    3.15681100e-04  -3.55590641e-01   2.64485270e-01  -7.65561163e-01
    5.91937125e-01  -6.36958122e-01]
 [ -9.41289127e-01  -9.77757931e-01   5.13038278e-01   6.21540427e-01
   -6.01109229e-02  -9.54740644e-02  -9.09503758e-01   8.76764297e-01
    4.35540587e-01  -6.08670294e-01]] [[ 0.77826536]
 [ 0.0264528 ]
 [-0.57920176]
 [-0.73537982]
 [ 0.4157536 ]
 [ 0.53630739]
 [ 0.74713212]
 [ 0.76174855]
 [-0.89140159]
 [ 0.12504567]]
4000 0.0475369 [[-0.67901284 -3.75251913  0.91760194  0.41951516 -0.18584128 -0.56711423
   3.88454723 -5.20162582  0.71657544 -2.03415203]
 [-0.69072241 -3.75404263  0.39696571  0.96043807 -0.34797281 -0.349581
  -5.17144775  3.9824934   0.69583714 -1.99820197]] [[ 0.41964954]
 [-4.36814499]
 [-1.07985377]
 [-1.46920991]
 [ 0.3952693 ]
 [ 0.66347247]
 [ 6.51861477]
 [ 6.55287504]
 [-1.59711707]
 [-1.6419307 ]]
8000 0.0139586 [[-0.71343887 -4.25359392  1.07601202  0.51918066 -0.

## Deep NN for XOR

In [10]:
# Deep network configuration.: Use more layers. 
W1 = tf.Variable(tf.random_uniform( [2,5], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [5,4], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform( [4,1], -1.0, 1.0))


b1 = tf.Variable(tf.zeros([5]), name="Bias1")
b2 = tf.Variable(tf.zeros([4]), name="Bias2")
b3 = tf.Variable(tf.zeros([1]), name="Bias3")

In [11]:
# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
L3 =  tf.sigmoid(tf.matmul(L2,W2)+b2)
hypothesis = tf.sigmoid( tf.matmul(L3,W3) + b3)

In [12]:
# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.global_variables_initializer()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 4000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.700846 [[ 0.39776748 -0.22293001  0.3412393  -0.57779938 -0.6393913 ]
 [ 0.69097     0.65415448 -0.28799036 -0.02191218 -0.89048553]] [[ 0.76226175 -0.06589856 -0.29528084  0.29179454]
 [-0.94714946  0.11760885  0.18141089 -0.95753485]
 [-0.92602921  0.67295933 -0.54780787 -0.05619263]
 [ 0.21244694 -0.16301087  0.63588417 -0.57583523]
 [-0.79140913  0.63127655  0.15852229  0.41589442]]
4000 0.691782 [[ 0.45119584 -0.38180345  0.40844983 -0.63317966 -0.63092858]
 [ 0.68571591  0.84390891 -0.28297582  0.03112993 -0.93557441]] [[ 0.74379474 -0.14752619 -0.21970776  0.34231362]
 [-0.95118713  0.11790286  0.14838721 -1.12224054]
 [-0.93212599  0.67778522 -0.56667155 -0.04573681]
 [ 0.19510946 -0.21197037  0.62882167 -0.66049933]
 [-0.77089345  0.76783371 -0.03541021  0.16074578]]
8000 0.141503 [[ 1.82208431 -2.80125093  2.66344547 -2.29024553 -3.15621352]
 [ 2.06240916  4.22189474 -1.09472704  0.70652229 -3.47796702]] [[ 1.68940616 -1.23733091  0.09898234  0.98444045]
 [-2.14720321  1.